In [ ]:
#Importar librerias
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from scipy.stats import uniform
from sklearn.metrics import  make_scorer
from sklearn.utils.fixes import loguniform
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.model_selection import validation_curve

In [ ]:
#leer el dataframe. Poner el normal
dataframe=pd.read_csv('/content/97-fosfo-343.csv')
dataframe = dataframe.drop(['id', 'aislamiento', 'FOSFOI'], axis='columns')
#defino las variables "X" y "y"
X = dataframe.drop(['FOSFOMICINA_'], axis='columns')
y = dataframe.iloc[:, -1]  
ylog=np.log2(y)

In [ ]:
#perform permutation importance
modelo = RandomForestRegressor()
modelo.fit(X, ylog)
characteristic = dataframe.columns
results = permutation_importance(modelo, X, ylog, scoring='neg_mean_squared_error')
# get importance
importances = results.importances_mean
characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristic, importances)]
characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances];


Variable: Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin_cp1 Importance: 2.63
Variable: AAC(6')-Ian          Importance: 0.76
Variable: ugd                  Importance: 0.33
Variable: cmlB1                Importance: 0.23
Variable: TEM-1                Importance: 0.2
Variable: OXA-65               Importance: 0.18
Variable: AAC(3)-IIe           Importance: 0.11
Variable: OXA-64               Importance: 0.1
Variable: adeS                 Importance: 0.1
Variable: OXA-146              Importance: 0.1
Variable: adeN                 Importance: 0.07
Variable: adeL                 Importance: 0.06
Variable: ADC-5                Importance: 0.06
Variable: Mycobacterium_tuberculosis_rpoB_mutants_conferring_resistance_to_rifampicin Importance: 0.04
Variable: APH(6)-Id            Importance: 0.03
Variable: ADC-2                Importance: 0.03
Variable: Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin Importance: 0.03
Variable: adeA                 Import

In [ ]:
#leer dataframe para dummys 
dataframe=pd.read_csv('/content/97-tige-343dummys.csv')
dataframe = dataframe.drop(['id', 'aislamiento', 'TIGEI'], axis='columns')
#defino las variables "X" y "y"
X = dataframe.drop(['TIGECICLINA'], axis='columns')
y = dataframe.iloc[:, -1]  
ylog=np.log2(y)

In [ ]:
#funcion para calcular la excatitud con +/- una dilucion 
def within_dillution(y_test, predictions, dill=2):
    frac = np.array(y_test) / np.array(predictions)
    return ((frac >= 1/dill) & (frac <= dill)).mean()

In [ ]:
genes=[]
linea=[]
resultados=[]
for pair in characteristics_importances:#[:final]
  linea=[]
  genes.append(pair[0])
  #print(genes)
  X = dataframe[genes]
  #cambio de variables a categoricas por dummys 
  X_dummys = pd.get_dummies(X)
  #print(X_dummys)
  #Eliminar columnas que indican la ausencia del gen 
  for col in X_dummys.columns: 
    if col[-3:] == 'ivo':
      print(col)
      del(X_dummys[col])
  print(X_dummys.columns)
  print(X_dummys.shape)
  #divido en train y test
  X_train, X_test, y_train, y_test = train_test_split(X_dummys, y, test_size=0.20, random_state=27)
  print(X_train)
  #convierto la variable "y" en log 
  y_train1 = np.log2(y_train)
  y_test1 = np.log2(y_test)
  print(y_train1)
  #ajuste de hiperparametros por gridesearch
  # find optimal alpha with grid search: REGULARIZACION
  lasso = Lasso()
  alpha = [0.001, 0.005, 0.01, 0.05, 0.06, 0.07, 0.08, 0.085, 0.086, 0.087, 0.088, 0.089, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 10, 100, 1000]
  param_grid = dict(alpha=alpha)
  grid = GridSearchCV(estimator=lasso, param_grid=param_grid, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1, cv=5)
  grid_result = grid.fit(X_train, y_train1)
  print('Best Score: ', grid_result.best_score_)
  print('Best Params: ', grid_result.best_params_)
  linea.append(grid_result.best_params_)
  #hacer regularizacion lasso
  #regularizacion lazo utilizando otro valor de alpha 
  lasso001 = Lasso(alpha= grid.best_params_['alpha'], max_iter=10e5)
  lasso001.fit(X_train, y_train1)
  train_score = lasso001.score(X_train, y_train1)
  test_score = lasso001.score(X_test, y_test1)
  coeff_used = np.sum(lasso001.coef_!=0)
  print("train score:", train_score) #r2 del entrenamiento 
  print("test score:", test_score) #r2 del test
  print("numero de caracteristicas usadas:", coeff_used)
  #hacer prediccion 
  predictions = 2**np.round(lasso001.predict(X_test))
  y_pred = np.log2(predictions)
  verdaderos = y_test.to_numpy()
  #obtengo los valores que muestran el desempeño del modelo
  maete=metrics.mean_absolute_error(y_test1, y_pred)
  print('Mean Absolute Error_test:', maete)
  linea.append(maete)
  maetr=metrics.mean_absolute_error(y_train1, lasso001.predict(X_train))
  print('Mean Absolute Error_train:', maetr)
  linea.append(maetr)
  msete=metrics.mean_squared_error(y_test1, y_pred)
  print('Mean Squared Error_test:', msete)
  linea.append(msete)
  msetr=metrics.mean_squared_error(y_train1, lasso001.predict(X_train))
  print('Mean Squared Error_train:', msetr)
  linea.append(msetr)
  rmsete=np.sqrt(metrics.mean_squared_error(y_test1, y_pred))
  print('Root Mean Squared Error_test:', rmsete)
  linea.append(rmsete)
  rmstr=np.sqrt(metrics.mean_squared_error(y_train1, lasso001.predict(X_train)))
  print('RMSE_train:', rmstr)
  linea.append(rmstr)
  r2tr=metrics.r2_score(y_train1, lasso001.predict(X_train))
  print('r2_train:', r2tr)
  linea.append(r2tr)
  r2te=metrics.r2_score(y_test1, (y_pred))
  print('r2_teste:', r2te)
  linea.append(r2te)
  #saco accuracy con +/- 1 dilucion, VME y ME
  acc2 = within_dillution(y_test, predictions, dill=2)
  acc4 = within_dillution(y_test, predictions, dill=4)
  threshold = 1
  threshold1 = 2
  predDF = pd.DataFrame({'predictions':predictions, 'y_test':y_test})
  vme = len(predDF[(predDF.predictions<=threshold) & (predDF.y_test>threshold1)])
  me = len(predDF[(predDF.predictions>threshold1) & (predDF.y_test<=threshold)])
  print(vme)
  print(me)
  #print(vme)
  linea.append(vme)
  #print(me)
  linea.append(me)
  #print(acc2)
  linea.append(acc2)
  #print(acc4)
  resultados.append(linea)

adeC_negativo
Index(['adeC_cinco', 'adeC_cuatro', 'adeC_seis', 'adeC_tres', 'adeC_uno'], dtype='object')
(243, 5)
     adeC_cinco  adeC_cuatro  adeC_seis  adeC_tres  adeC_uno
200           0            0          1          0         0
106           0            0          0          0         1
144           0            0          0          0         1
209           0            0          0          0         0
160           0            0          0          0         0
..          ...          ...        ...        ...       ...
56            0            0          0          0         0
31            0            0          0          0         0
72            0            0          0          0         0
184           0            0          0          0         1
19            0            0          0          0         0

[194 rows x 5 columns]
200   -2.0
106    0.0
144    0.0
209    1.0
160   -2.0
      ... 
56    -2.0
31    -3.0
72    -2.0
184    0.0
19    -2.0
Name: TIG

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best Score:  -1.0189449547490583
Best Params:  {'alpha': 0.001}
train score: 0.41479177204470136
test score: 0.2556337242529808
numero de caracteristicas usadas: 5
Mean Absolute Error_test: 0.8571428571428571
Mean Absolute Error_train: 0.7249033176259174
Mean Squared Error_test: 1.5918367346938775
Mean Squared Error_train: 0.9793483640540049
Root Mean Squared Error_test: 1.261680123761121
RMSE_train: 0.9896203130766895
r2_train: 0.4147917720447014
r2_teste: 0.07992296581608094
0
0
adeC_negativo
sul1_negativo
Index(['adeC_cinco', 'adeC_cuatro', 'adeC_seis', 'adeC_tres', 'adeC_uno',
       'sul1_uno'],
      dtype='object')
(243, 6)
     adeC_cinco  adeC_cuatro  adeC_seis  adeC_tres  adeC_uno  sul1_uno
200           0            0          1          0         0         0
106           0            0          0          0         1         1
144           0            0          0          0         1         1
209           0            0          0          0         0         0
160   

[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best Score:  -1.004594710077394
Best Params:  {'alpha': 0.001}
train score: 0.43507094398739116
test score: 0.26742276773058704
numero de caracteristicas usadas: 6
Mean Absolute Error_test: 0.8571428571428571
Mean Absolute Error_train: 0.7230005094483741
Mean Squared Error_test: 1.5918367346938775
Mean Squared Error_train: 0.9454110868290508
Root Mean Squared Error_test: 1.261680123761121
RMSE_train: 0.9723225220208832
r2_train: 0.4350709439873911
r2_teste: 0.07992296581608094
0
0
adeC_negativo
sul1_negativo
Escherichia_coli_gyrA_conferring_resistance_to_fluoroquinolones_negativo
Index(['adeC_cinco', 'adeC_cuatro', 'adeC_seis', 'adeC_tres', 'adeC_uno',
       'sul1_uno',
       'Escherichia_coli_gyrA_conferring_resistance_to_fluoroquinolones_uno'],
      dtype='object')
(243, 7)
     adeC_cinco  ...  Escherichia_coli_gyrA_conferring_resistance_to_fluoroquinolones_uno
200           0  ...                                                  0                  
106           0  ...          

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed:    0.5s finished


Best Score:  -0.9893264516801772
Best Params:  {'alpha': 0.01}
train score: 0.6600137848326697
test score: 0.275295390842776
numero de caracteristicas usadas: 44
Mean Absolute Error_test: 0.6938775510204082
Mean Absolute Error_train: 0.5044531874130166
Mean Squared Error_test: 1.2244897959183674
Mean Squared Error_train: 0.5689683222472934
Root Mean Squared Error_test: 1.1065666703449764
RMSE_train: 0.7542998888023871
r2_train: 0.6600137848326697
r2_teste: 0.29224843524313926
0
0
adeC_negativo
sul1_negativo
Escherichia_coli_gyrA_conferring_resistance_to_fluoroquinolones_negativo
APH(3'')-Ib_negativo
adeS_negativo
Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin_cp1_negativo
APH(3')-VIa_negativo
MexK_negativo
ugd_negativo
ANT(2'')-Ia_negativo
OXA-64_negativo
Acinetobacter_baumannii_AmvA_negativo
adeN_negativo
adeL_negativo
AAC(6')-Ian_negativo
TEM-1_negativo
OXA-90_negativo
mphE_negativo
OXA-72_negativo
APH(3')-VI_negativo
AAC(6')-Iaf_negativo
abeS_negativo
ANT(3'')-II

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best Score:  -0.9893264516801772
Best Params:  {'alpha': 0.01}
train score: 0.6600137848326697
test score: 0.275295390842776
numero de caracteristicas usadas: 44
Mean Absolute Error_test: 0.6938775510204082
Mean Absolute Error_train: 0.5044531874130166
Mean Squared Error_test: 1.2244897959183674
Mean Squared Error_train: 0.5689683222472934
Root Mean Squared Error_test: 1.1065666703449764
RMSE_train: 0.7542998888023871
r2_train: 0.6600137848326697
r2_teste: 0.29224843524313926
0
0


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed:    0.5s finished


In [ ]:
resultadosdf=pd.DataFrame(resultados)
print(resultadosdf)
resultadosdf.to_excel ('TIGE-97-RL-PERM.xlsx', index = None, header=True)

                   0         1         2         3   ...        8   9   10        11
0    {'alpha': 0.001}  0.857143  0.724903  1.591837  ...  0.079923   0   0  0.836735
1    {'alpha': 0.001}  0.857143  0.723001  1.591837  ...  0.079923   0   0  0.836735
2    {'alpha': 0.001}  0.734694  0.691815  1.510204  ...  0.127106   0   0  0.816327
3    {'alpha': 0.001}  0.734694  0.693447  1.510204  ...  0.127106   0   0  0.816327
4     {'alpha': 0.01}  0.755102  0.702404  1.489796  ...  0.138902   0   0  0.836735
..                ...       ...       ...       ...  ...       ...  ..  ..       ...
206   {'alpha': 0.01}  0.693878  0.504453  1.224490  ...  0.292248   0   0  0.857143
207   {'alpha': 0.01}  0.693878  0.504453  1.224490  ...  0.292248   0   0  0.857143
208   {'alpha': 0.01}  0.693878  0.504453  1.224490  ...  0.292248   0   0  0.857143
209   {'alpha': 0.01}  0.693878  0.504453  1.224490  ...  0.292248   0   0  0.857143
210   {'alpha': 0.01}  0.693878  0.504453  1.224490  ...  0.29224

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
